In [1]:
# -*- config : utf-8 -*-
import numpy as np
import tensorflow as tf
import sys
import time
import warnings

# 경고 메시지 무시
warnings.filterwarnings(action='ignore')

In [2]:
# --- 설정 변수 정의 ---
# 작업 디렉토리 정의
# 학습 데이터 파일명
noise_list_file = "/src/hyebin/esnoise/noise_spec_data.npy"
# 라벨 데이터 파일명
noise_label_list_file = "/src/hyebin/esnoise/noise_label_data.npy"
# 진동 분석 모델 디렉토리명
modeldir = "/src/hyebin/model/NOISE-SPEC-01"
# tensorboard 용 log 디렉토리명
tblogdir = "/src/hyebin/logs"

# 학습 데이터 설정
# 학습할 데이터의 개수
training_count = 2000

# 전결합층 설정
# 전결합층 첫번째 레이어 unit 개수
layer1_unit_count = 1024
# 전결합층 두번째 레이어 unit 개수
layer2_unit_count = 1024
# 전결합층 세번째 레이어 unit 개수
layer3_unit_count = 1024
# 전결합층 dropout 비율
dropout_rate = 0.4

# 최적화 함수 설정
# 최적화 함수 학습률
learning_rate = 0.0001

# 학습 수행 설정
# 최대 학습 수행 횟수
epoch_count = 5000
# 학습 중단 시킬, 최소 loss값
loss_limit = 0.0001

In [3]:
# --- 학습 데이터 로딩 ---
# 데이터 구조
# noise_list : 학습용 데이터(이미지 개수, 48, 64, 3)
# noise_label_list : 학습용 라벨 데이터(이미지 개수, 18)
print("load noise data:", noise_list_file)
noise_list = np.load(noise_list_file, allow_pickle=True)
print("load complete")

print("load noise label data:", noise_label_list_file)
org_noise_label_list = np.load(noise_label_list_file, allow_pickle=True)

# one hot encoding
noise_label_list = []
unit_matrix = np.identity(18)
for index in org_noise_label_list:
    one_hot = unit_matrix[index:index+1][0]
    noise_label_list.append(one_hot)

noise_label_list = np.array(noise_label_list)

print("load complete")

print(noise_list.shape)
print(noise_label_list.shape)

load noise data: /src/hyebin/esnoise/noise_spec_data.npy
load complete
load noise label data: /src/hyebin/esnoise/noise_label_data.npy
load complete
(18000, 48, 64, 3)
(18000, 18)


In [4]:
# --- VGG Net 신경망 구성 ---
#     입력층 : 48*64 의 Spectrogram 전처리 데이터
#     특징 추출층 : 2개 합성곱 필터, 1개의 필터를 2회 수행 
#     전 결합층 : 3개의 은닉층(층당 유닛 1024개) → 단층에서 발생하는 xor 문제 회피, 
#                1개의 출력층(유닛 5개)

tf.reset_default_graph()

# dropout 사용 여부
dropout_rate = tf.placeholder_with_default(0, shape=[], name="dropout_rate")

# --- 입력층 ---
# Spectrogram
x = tf.placeholder(tf.float32, [None, 48, 64, 3], name="in")

# --- 특징 추출층 ---
conv11 = tf.layers.conv2d(
    inputs=x,
    filters=32,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu)

conv12 = tf.layers.conv2d(
    inputs=conv11,
    filters=32,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu)

# 풀링으로 사이즈가 줄어듦
# Spectrogram : 48 * 64 → 24 * 32
pool1 = tf.layers.max_pooling2d(
    inputs=conv12,
    pool_size=[2, 2],
    strides=2)

conv21 = tf.layers.conv2d(
    inputs=pool1,
    filters=64,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu)

conv22 = tf.layers.conv2d(
    inputs=conv21,
    filters=64,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu)

# 풀링으로 사이즈가 줄어듦
# Spectrogram : 24 * 32 → 12 * 16
pool2 = tf.layers.max_pooling2d(
    inputs=conv22,
    pool_size=[2, 2],
    strides=2)

conv31 = tf.layers.conv2d(
    inputs=pool2,
    filters=128,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu)

conv32 = tf.layers.conv2d(
    inputs=conv31,
    filters=128,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu)

# 풀링으로 사이즈가 줄어듦
# Spectrogram : 12 * 16 → 6 * 8
pool3 = tf.layers.max_pooling2d(
    inputs=conv32,
    pool_size=[2, 2],
    strides=2)

# --- 전 결합층 ---
# 데이터 행렬 평탄화(flatten)
# Spectrogram : 6 * 8(이미지사이즈) * 128(합성곱 필터 수)
input_cnt = 6 * 8 * 128
pool_flat = tf.reshape(pool3, [-1, input_cnt])

dense1 = tf.layers.dense(inputs=pool_flat, units=layer1_unit_count, activation=tf.nn.relu)
dropout1 = tf.layers.dropout(inputs=dense1, rate=dropout_rate)

dense2 = tf.layers.dense(inputs=dropout1, units=layer2_unit_count, activation=tf.nn.relu)
dropout2 = tf.layers.dropout(inputs=dense2, rate=dropout_rate)

dense3 = tf.layers.dense(inputs=dropout2, units=layer3_unit_count, activation=tf.nn.relu)
dropout3 = tf.layers.dropout(inputs=dense3, rate=dropout_rate)

netout = tf.layers.dense(inputs=dropout3, units=18)
softout = tf.nn.softmax(netout, name="out")

# --- loss 함수 정의 ---
# 평균 제곱 오차 함수 사용
y = tf.placeholder(tf.float32, [None, 18])
loss = tf.reduce_mean(tf.reduce_sum(tf.square(softout - y), 1))

print(type(loss))
print('loss : {}'.format(loss))

loss_summ = tf.summary.scalar("loss", loss)

# --- loss 함수의 최적화 함수 정의 ---
train = tf.train.AdamOptimizer(learning_rate).minimize(loss)

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Use keras.layers.dropout instead.
<class 'tensorflow.python.framework.ops.Tensor'>
loss : Tensor("Mean:0", shape=(), dtype=float32)
<class 'tensorflow.python.framework.ops.Operation'>
train : name: "Adam"
op: "NoOp"
input: "^Adam/update_conv2d/kernel/ApplyAdam"
input: "^Adam/update_conv2d/bias/ApplyAdam"
input: "^Adam/update_conv2d_1/kernel/ApplyAdam"
input: "^Adam/update_conv2d_1/bias/ApplyAdam"
input: "^Adam/update_conv2d_2/kernel/ApplyAdam"
input: "^Adam/update_conv2d_2/bias/ApplyAdam"
input: "^Adam/update_conv2d_3/kernel/ApplyAdam"
input: "^Adam/update_conv2d_3/bias/ApplyAdam"
input: "^Adam/update_conv2d_4/kernel/ApplyAdam"
input: "^Adam/update_conv2d_4/bias/ApplyAdam"
input: "^Adam/update_conv2d_5/

In [5]:
# --- 학습 수행 및 모델 저장---
with tf.Session() as sess:

    # tensorboard 용 로깅 정의    
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter(tblogdir)
    writer.add_graph(sess.graph)
    
    # 신경망 가중치(weight) 초기화
    sess.run(tf.global_variables_initializer())
    
    # 전체 데이터에 대해 5000번 수행
    # loss가 0.001 이하일때, 학습 중단
    start_time = time.time()
    for epoch in range(epoch_count):
        
        # 학습용 데이터와 라벨
        # 랜덤 데이터를 training_count 만큼 가져옴
        base_index = np.random.randint(noise_label_list.shape[0] - training_count)
        train_x = noise_list[base_index:base_index + training_count]
        train_y = noise_label_list[base_index:base_index + training_count]

        # 학습 수행
        _train, _loss_summ, _loss = sess.run([train, loss_summ, loss], feed_dict={x:train_x, y:train_y, dropout_rate:0.4})
        writer.add_summary(summary = _loss_summ, global_step = epoch)
        
        if epoch % 100 == 0:
            print("index=", epoch, "  loss=", _loss)
            
        if _loss < loss_limit:
            print("index=", epoch, "  loss=", _loss)
            break
            
    end_time = time.time()
    print("running time:", end_time - start_time)
    
    # --- 학습 모델 저장 ----
    print("save model")
    builder = tf.saved_model.builder.SavedModelBuilder(modeldir)
    signature = tf.saved_model.predict_signature_def(inputs={"in":x}, outputs={"out":softout})
    builder.add_meta_graph_and_variables(sess, tags=["ver1"], signature_def_map={"escalator-vibration-analysis-v1":signature})
    builder.save()
    print("complete")

index= 0   loss= 0.9444577
index= 100   loss= 0.54797024
index= 200   loss= 0.10793387
index= 300   loss= 0.02527571
index= 400   loss= 0.013466772
index= 500   loss= 0.0048711714
index= 600   loss= 0.0050850874
index= 700   loss= 0.0022025933
index= 800   loss= 0.0011179049
index= 900   loss= 0.00029893633
index= 1000   loss= 0.00037744085
index= 1100   loss= 0.00015496383
index= 1119   loss= 9.948237e-05
running time: 532.6603753566742
save model


AssertionError: Export directory already exists, and isn't empty. Please choose a different export directory, or delete all the contents of the specified directory: /src/hyebin/model/NOISE-SPEC-01